In [1]:
import pandas as pd
import numpy as np
import datetime
import math
from gensim import corpora, models
from pyltp import SentenceSplitter
import functools
import os
from pyltp import Segmentor
from pyltp import Postagger


In [2]:
all_docs_df = pd.read_csv('../../data/chusai/all_docs.txt', sep='\001', header=None)
all_docs_df.columns = ['id', 'title', 'text']
all_docs_df['title'] = all_docs_df['title'].astype(str)
all_docs_df['text'] = all_docs_df['text'].astype(str)
print(all_docs_df.info())
print(all_docs_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108295 entries, 0 to 108294
Data columns (total 3 columns):
id       108295 non-null object
title    108295 non-null object
text     108295 non-null object
dtypes: object(3)
memory usage: 2.5+ MB
None
        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...  
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...  
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...  
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...  
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...  


In [3]:
train_doc_keyword_df = pd.read_csv('../../data/chusai/train_docs_keywords.txt', sep='\t', header=None)
train_doc_keyword_df.columns = ['id', 'keyword']
train_doc_keyword_df['keyword_list'] = train_doc_keyword_df['keyword'].map(lambda x: x.split(','))
print(train_doc_keyword_df.head(10))


        id            keyword           keyword_list
0  D012650         三生三世,凤九,东华         [三生三世, 凤九, 东华]
1  D047118              南宁,美食               [南宁, 美食]
2  D098970             凉菜,北京人              [凉菜, 北京人]
3  D092010          华为,P30pro           [华为, P30pro]
4  D103408             酒吧,世界杯              [酒吧, 世界杯]
5  D103809            烧烤,人生一串             [烧烤, 人生一串]
6  D003823         惠若琪,杨颖,奔跑吧         [惠若琪, 杨颖, 奔跑吧]
7  D023499  我与你的光年距离2,王以纶,许晓诺  [我与你的光年距离2, 王以纶, 许晓诺]
8  D063937          无人机,军用,美军          [无人机, 军用, 美军]
9  D059085           女兵,剪影,欧美           [女兵, 剪影, 欧美]


In [4]:
LTP_DATA_DIR = '../../ltp/ltp_data_v3.4.0/'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型


In [6]:
#停用词表加载方法
def get_stopword_list():
    #停用词表存储路径，每一行为一个词，按行读取进行加载
    #进行编码转换确保匹配准确率
    stop_word_path = '../stopword.txt'
    stop_word_list = [sw.replace('\n', '') for sw in open(stop_word_path).readlines()]
    return stop_word_list

#分词方法，调用ltp接口
def seg_to_list(sentence):
    words = segmentor.segment(sentence)  # 分词
    seg_list = list(words)
    return seg_list

#去除干扰词
def word_filter(seg_list, stopword_list):
    
    filter_list = []
    #根据pos参数选择是否词性过滤
    #不进行词性过滤，则将词性都标记为n，表示全部保留
    for seg in seg_list:
        #过滤高停用词表中的词，以及长度为<2的词
        if not seg in stopword_list and len(seg) > 1:
            filter_list.append(seg)
    
    return filter_list

def jieba_word_deal(sentence, stopword_list):
    #调用上面方式对数据集进行处理，处理后的每条数据仅保留非干扰词
    seg_list = seg_to_list(sentence)
    filter_list = word_filter(seg_list, stopword_list)
    return filter_list

stopword_list = get_stopword_list()
all_docs_df['title_list'] = all_docs_df['title'].map(lambda x : jieba_word_deal(x, stopword_list))
all_docs_df['text_list'] = all_docs_df['text'].map(lambda x : jieba_word_deal(x, stopword_list))
all_docs_df['title_text'] = all_docs_df['title'] + '。' + all_docs_df['text']
all_docs_df['title_text_list'] = all_docs_df['title_text'].map(lambda x : jieba_word_deal(x, stopword_list))
print(all_docs_df.head(10))


        id                   title  \
0  D000001      林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002      小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003       年轻时的她风华绝代，现却无人送祝福   
3  D000004      林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005     曾是TVB颜值担当，近照曝光发现真老了   
5  D000006  16岁的范冰冰有多美？塞娅公主看一眼被惊艳到   
6  D000007        一样的北京，不一样的周迅与周冬雨   
7  D000008      张杰自嘲是废物，赞谢娜是新时代好女性   
8  D000009    杨幂获杂志主编夸奖：大幂幂瘦了越来越漂亮   
9  D000010       杨幂女扮男装现身新节目瞬间年轻十岁   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   
5  范冰冰，家喻户晓的人物了，在出演还珠格格之前已经亭亭玉立十分美丽，在一些影剧中饰演一些小角色...   
6  不知道刘若英与陈可辛之间有怎样的交情。《后来的我们》首映请来了陈可辛，他看完后笑着说很羡慕刘...   
7  本文是企鹅号"不八卦会死人"原创文章，独家发布企鹅号，未经允许，不得转载!最新一期《向往的生...   
8  网易娱乐4月29日报道  4月29日，时尚主编张宇晒出一组与杨幂的自拍合影，并发文：“今天与...   
9  杨幂前段时间录制的综艺节目《我是大侦

In [15]:
all_docs_df['title_word_attributes'] = all_docs_df['title_list'].map(lambda x: list(postagger.postag(x)))
print(all_docs_df.head())



        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                  title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 吓人]   
1          [夸杨幂, 身材, 回复, 精彩]   
2             [年轻, 风华绝代, 祝福]   
3  [林心, 霍建华, 生活, 怼蹭, 老公, 人气]   
4  [TVB, 颜值, 担当, 近照, 曝光, 发现]   

                                           text_list  \
0  [早年, 志颖, kimi, 爸爸, 当时, 遮遮掩掩, 志颖, 老婆, 低调, 探班, 觉...   
1  [翩若, 惊鸿, 婉若, 游龙, 曹植, 形容, 洛神, 实在, 抽象, 始终, 承受, 对...   
2  [世纪, 香港, 影视界, 涌现, 不少, 高颜值, 女星, 大话, 西游, 光宝盒, 饰演...   
3  [霍建华, 心如, 1905, 电影, 网讯, 近日, 林心如,

In [18]:
def get_title_person_name(title_list):
    attributes_list = list(postagger.postag(title_list))
    title_person_name_list = list()
    i = 0
    for attributes in attributes_list:
        if attributes == 'nh':
            title_person_name_list.append(title_list[i])
        i = i + 1
    return title_person_name_list

all_docs_df['title_person_name_list'] = all_docs_df['title_list'].map(lambda x: get_title_person_name(x))
print(all_docs_df[['title_list', 'title_person_name_list']].head(50))


                                     title_list     title_person_name_list
0                     [林志颖, 老婆, 深夜, 面膜, 睫毛, 吓人]                      [林志颖]
1                             [夸杨幂, 身材, 回复, 精彩]                         []
2                                [年轻, 风华绝代, 祝福]                         []
3                     [林心, 霍建华, 生活, 怼蹭, 老公, 人气]                  [林心, 霍建华]
4                     [TVB, 颜值, 担当, 近照, 曝光, 发现]                         []
5                          [16, 冰冰, 塞娅, 公主, 惊艳]                       [塞娅]
6                                 [北京, 周迅, 周冬雨]                  [周迅, 周冬雨]
7                     [张杰, 自嘲, 废物, 赞谢娜, 时代, 女性]                       [张杰]
8                    [杂志, 主编, 夸奖, 大幂幂, 越来越, 漂亮]                         []
9                          [男装, 现身, 节目, 瞬间, 年轻]                         []
10                     [搭戏, 彦祖, 出名, 如今, 40, 迎来]                         []
11                   [尴尬, 灏明, 音乐节, 卫生间, 突然, 拉开]                         []
12                       

In [7]:
#idf统计方法
def train_idf(doc_list):
    idf_dic = {}
    #总文档数
    tt_count = len(doc_list)
    
    #每个词出现的文档数
    for doc in doc_list:
        for word in set(doc):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
    
    #按公式转换为idf值，分母加1进行平滑处理
    for k, v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (1.0 + v))
        
    #对于没有在字典中的词，默认其尽在一个文档中出现，得到默认idf值
    default_idf = math.log(tt_count / (1.0))
    return idf_dic, default_idf

#排序函数，用于topK关键词的按值排
def cmp(e1, e2):
    res = np.sign(e1[1] - e2[1])
    if res != 0:
        return res
    else:
        a = e1[0] + e2[0]
        b = e2[0] + e1[0]
        if a > b:
            return 1
        elif a == b:
            return 0
        else:
            return -1


In [8]:
#TF-IDF类
class TfIdf(object):
    #四个参数分别是：训练好的idf字典，默认idf值，处理后的待提取文本，关键词数量
    def __init__(self, idf_dic, default_idf, word_list, keyword_num):
        self.word_list = word_list
        self.idf_dic = idf_dic
        self.default_idf = default_idf
        self.tf_dic = self.get_tf_dic()
        self.keyword_num = keyword_num
    
    #统计tf值
    def get_tf_dic(self):
        tf_dic = {}
        for word in self.word_list:
            tf_dic[word] = tf_dic.get(word, 0.0) + 1.0
        
        tt_count = len(self.word_list)
        for k, v in tf_dic.items():
            tf_dic[k] = float(v) / tt_count
        
        return tf_dic
    
    #按公式计算tf-idf
    def get_tfidf(self):
        tfidf_dic = {}
        for word in self.word_list:
            idf = self.idf_dic.get(word, self.default_idf)
            tf = self.tf_dic.get(word, 0)
            
            tfidf = tf * idf
            tfidf_dic[word] = tfidf
        
        result_dict = {}
        for k, v in sorted(tfidf_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
            result_dict[k] = result_dict.get(k, 0.0) + float(v)
        return result_dict





In [9]:
idf_dic, default_idf = train_idf(all_docs_df['title_text_list'])
keyword_num = 5
all_docs_df['result_dict'] = all_docs_df['title_text_list'].map(lambda x: TfIdf(idf_dic, default_idf, x, keyword_num).get_tfidf())
print(all_docs_df.head())



        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                  title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 吓人]   
1          [夸杨幂, 身材, 回复, 精彩]   
2             [年轻, 风华绝代, 祝福]   
3  [林心, 霍建华, 生活, 怼蹭, 老公, 人气]   
4  [TVB, 颜值, 担当, 近照, 曝光, 发现]   

                                           text_list  \
0  [早年, 志颖, kimi, 爸爸, 当时, 遮遮掩掩, 志颖, 老婆, 低调, 探班, 觉...   
1  [翩若, 惊鸿, 婉若, 游龙, 曹植, 形容, 洛神, 实在, 抽象, 始终, 承受, 对...   
2  [世纪, 香港, 影视界, 涌现, 不少, 高颜值, 女星, 大话, 西游, 光宝盒, 饰演...   
3  [霍建华, 心如, 1905, 电影, 网讯, 近日, 林心如,

In [10]:
sample_df = pd.read_csv('../../result/chusai/sample.csv', encoding='ISO-8859-1')
print(len(sample_df))
sample_df = pd.merge(sample_df, all_docs_df, on='id', how='left')
print(sample_df.head())


107295
        id label1 label2                title  \
0  D000001     ÎÞ     ÎÞ   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002     ÎÞ     ÎÞ   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003     ÎÞ     ÎÞ    年轻时的她风华绝代，现却无人送祝福   
3  D000004     ÎÞ     ÎÞ   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005     ÎÞ     ÎÞ  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                  title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 吓人]   
1          [夸杨幂, 身材, 回复, 精彩]   
2             [年轻, 风华绝代, 祝福]   
3  [林心, 霍建华, 生活, 怼蹭, 老公, 人气]   
4  [TVB, 颜值, 担当, 近照, 曝光, 发现]   

                                           text_list  \
0  [早年, 志颖, kimi, 爸爸, 当时, 遮遮掩掩, 志颖, 老婆, 低调, 探班, 觉...   
1  [翩若, 惊鸿, 婉若, 游龙, 曹植, 形容, 洛神, 实在, 抽象, 始终, 承受, 对...   


In [11]:
keyword_set = set()
for keyword_list in train_doc_keyword_df['keyword_list']:
    for keyword in keyword_list:
        keyword_set.add(keyword)

def get_deal_result_list(keyword_set, sample_df):
    temp_df = pd.DataFrame(columns=['id', 'result_list'])
    for temp_id, title_list, result_dict in sample_df[['id', 'title_list', 'result_dict']].values:
        title_set = set(title_list)
        keys = list(result_dict.keys())
        result_list = list()
        result_list_1 = list()
        result_list_2 = list()
        result_list_3 = list()
        result_list_4 = list()
        for title in title_set:
            if len(title) >= 5:
                result_list.append(title)
        for key in keys:
            if key in set(result_list):
                continue
            if((key in keyword_set) & (key in title_set)):
                result_list_1.append(key)
            else:
                if key in keyword_set:
                    result_list_2.append(key)
                else:
                    if key in title_set:
                        result_list_3.append(key)
                    else:
                        result_list_4.append(key)
        result_list = result_list + result_list_1 + result_list_3 + result_list_2 + result_list_4
        temp = pd.DataFrame([[temp_id, result_list]], columns=['id', 'result_list'])
        temp_df = pd.concat([temp_df, temp])
    print(temp_df.head())
    sample_df = pd.merge(sample_df, temp_df, on='id', how='left')
    return sample_df

def get_top_n_word(result_list, n):
    if len(result_list) < n:
        return '无'
    else:
        return result_list[n - 1]
    
sample_df = get_deal_result_list(keyword_set, sample_df)
sample_df['label1'] = sample_df['result_list'].map(lambda x: get_top_n_word(x, 1))
sample_df['label2'] = sample_df['result_list'].map(lambda x: get_top_n_word(x, 2))
print(sample_df.head())


        id              result_list
0  D000001    [睫毛, 面膜, 陈若仪, 安利, 日常]
0  D000002  [夸杨幂, 张钧甯, 洛神, 。翩若, 瓌姿]
0  D000003  [风华绝代, 生日, 妩媚, 蓝洁瑛, 有型]
0  D000004   [霍建华, 林心, 心如, 林心如, 索吻]
0  D000005   [TVB, 素颜, 姚莹莹, 药房, 剧集]
        id label1 label2                title  \
0  D000001     睫毛     面膜   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002    夸杨幂    张钧甯   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003   风华绝代     生日    年轻时的她风华绝代，现却无人送祝福   
3  D000004    霍建华     林心   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005    TVB     素颜  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                  title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 吓人]   
1          [夸杨幂, 身材, 回复, 精彩]   
2             [年轻, 风华绝代, 祝福]   
3  [林心, 霍建华, 生活, 怼蹭, 老公,

In [12]:
print(sample_df[['id', 'result_list', 'title_list']].head(50))


         id                      result_list  \
0   D000001            [睫毛, 面膜, 陈若仪, 安利, 日常]   
1   D000002          [夸杨幂, 张钧甯, 洛神, 。翩若, 瓌姿]   
2   D000003          [风华绝代, 生日, 妩媚, 蓝洁瑛, 有型]   
3   D000004           [霍建华, 林心, 心如, 林心如, 索吻]   
4   D000005           [TVB, 素颜, 姚莹莹, 药房, 剧集]   
5   D000006             [公主, 冰冰, 塞娅, 惊艳, 金锁]   
6   D000007           [周冬雨, 周迅, 小晓, 老孙, 林见东]   
7   D000008             [张杰, 自嘲, 废物, 劈柴, 企鹅]   
8   D000009           [主编, 少女味, 29日, 托腮, 张宇]   
9   D000010            [男装, 节目, 大幂幂, 预告, 看杨]   
10  D000011            [彦祖, 蒋怡, 篮球, 翁狄森, 球例]   
11  D000012            [甄嬛, 婚纱照, 剧外, 娘娘, 剧中]   
12  D000013            [志玲, 花娇, 壁花, 花仙子, 美腻]   
13  D000014           [主持人, 节目, 主持, 综艺, 撒贝宁]   
14  D000015        [沈春阳, 沈阳, 素颜长, 溜达, 风和日丽拍]   
15  D000016            [冠希, 陈冠希, 女儿, 滚君, 父亲]   
16  D000017          [张艺谋, 陈婷, 艺谋, 张伟平, 周晓枫]   
17  D000018            [超模, 刘雯, 模特, 模特界, 何穗]   
18  D000019         [祝绪丹, 饰演, 紫明宫, 白浅大, 唐楠楠]   
19  D000020            [漫威, 漫画, 英雄, DC, 

In [13]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../result/chusai/%s.csv' % fileName, header=True, index=False)
    
exportResult(sample_df[['id', 'label1', 'label2']], 'tfidf_9_5_3')
